## Using TopOpt's analysis

In [1]:
using TopOpt

In [2]:
ins_dir = joinpath(@__DIR__, "examples", "data")
file_name = "7_bar_truss.json"
problem_file = joinpath(ins_dir, file_name)

node_points, elements, mats, crosssecs, fixities, load_cases = load_truss_json(problem_file);
ndim, nnodes, ncells = length(node_points[1]), length(node_points), length(elements)
loads = load_cases["0"]

problem = TrussProblem(Val{:Linear}, node_points, elements, loads, fixities, mats, crosssecs);

In [3]:
problem.truss_grid.grid.nodes

5-element Vector{Ferrite.Node{2, Float64}}:
 Ferrite.Node{2, Float64}([0.0, 0.0])
 Ferrite.Node{2, Float64}([2.0, 0.0])
 Ferrite.Node{2, Float64}([4.0, 0.0])
 Ferrite.Node{2, Float64}([1.3, -1.0])
 Ferrite.Node{2, Float64}([2.7, -1.0])

In [34]:
problem.truss_grid.grid.cells

7-element Vector{Ferrite.Line2D}:
 Ferrite.Line2D((1, 2))
 Ferrite.Line2D((2, 3))
 Ferrite.Line2D((4, 5))
 Ferrite.Line2D((1, 4))
 Ferrite.Line2D((2, 4))
 Ferrite.Line2D((2, 5))
 Ferrite.Line2D((3, 5))

In [4]:
solver = FEASolver(Direct, problem)
solver()
solver.u;

In [9]:
include("./src/TrussOpt.jl")

varying_ids = [(4, [1,2])]
sfl = TrussOpt.TrussSumFL(problem)

TopOpt truss ∑ |F|L function


In [10]:
sfl([0.0, 0.01])

8000.2000000000035

In [12]:
using FiniteDifferences

In [14]:
x0 = [1.3, -1.0]
grad = FiniteDifferences.grad(central_fdm(5, 1), sfl, x0)[1]

2-element Vector{Float64}:
 12.000000000023583
 41.80000000010796

In [29]:
using Optim

In [139]:
using Nonconvex

In [56]:
lower_bounds = [0.0, -3.0]
upper_bounds = [2.0, 3.0];

In [254]:
# alg = SAMIN()
alg = GradientDescent()

# alg = BFGS()

GradientDescent{InitialPrevious{Float64}, HagerZhang{Float64, Base.RefValue{Bool}}, Nothing, Optim.var"#11#13"}(InitialPrevious{Float64}
  alpha: Float64 1.0
  alphamin: Float64 0.0
  alphamax: Float64 Inf
, HagerZhang{Float64, Base.RefValue{Bool}}
  delta: Float64 0.1
  sigma: Float64 0.9
  alphamax: Float64 Inf
  rho: Float64 5.0
  epsilon: Float64 1.0e-6
  gamma: Float64 0.66
  linesearchmax: Int64 50
  psi3: Float64 0.1
  display: Int64 0
  mayterminate: Base.RefValue{Bool}
, nothing, Optim.var"#11#13"(), Flat())

In [261]:
x0 = [1.0, 0.1]
# lower_bounds, upper_bounds
# ; 
results = Optim.optimize(sfl, lower_bounds, upper_bounds, x0, Fminbox(alg),
            Optim.Options(store_trace = true, extended_trace=true, iterations=100)) # autodiff = :forward

 * Status: success

 * Candidate solution
    Final objective value:     6.928203e+01

 * Found with
    Algorithm:     Fminbox with Gradient Descent

 * Convergence measures
    |x - x'|               = 5.59e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.80e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 2.35e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    31
    ∇f(x) calls:   31


In [262]:
using GLMakie
GLMakie.activate!()

# using Plots
# pyplot()

In [263]:
xs = LinRange(0.01, 1.9, 100)
ys = LinRange(-2, 2, 100)
zs = [sfl([x,y]) for x in xs, y in ys];

# x_grid = [x for x = xs for y = ys]
# y_grid = [y for x = xs for y = ys]
# z_grid = sfl.(zip(x_grid, y_grid));

In [264]:
x_history = [tr.metadata["x"][1] for tr in results.trace]
y_history = [tr.metadata["x"][2] for tr in results.trace]
fn_history = [tr.value for tr in results.trace];

In [265]:
fig = Figure()
ax = Axis3(fig[1, 1], title="7-bar truss", zlabel="∑|F|L", xlabel="node_x", ylabel="node_y")

Makie.surface!(xs, ys, zs)
Makie.lines!(x_history, y_history, fn_history, color=1:length(x_history), linewidth = 4, colormap=:cool) #, markersize = 40
# 
# Makie.text!("start", position=Makie.Point3f0(x_history[1], y_history[1], fn_history[1]), color=:yellow)

display(fig)

# plt = Plots.plot(x_grid, y_grid, z_grid, st = :surface, 
#      xlabel = "node_x", ylabel = "node_y", zlabel = "∑|F|L")
# Plots.scatter3d!(x_history, y_history, fn_history, markersize = 6, c = :orange)

GLMakie.Screen(...)

In [ ]:
# arrows(
#     ps, ns, fxaa=true, # turn on anti-aliasing
#     linecolor = :gray, arrowcolor = :black,
#     linewidth = 0.1, arrowsize = Vec3f0(0.3, 0.3, 0.4),
#     align = :center, axis=(type=Axis3,)
# )

# display()

# Constrained optimization

In [48]:
using LinearAlgebra

nodes = problem.truss_grid.grid.nodes
axis_node = nodes[2]
cells = problem.truss_grid.grid.cells

length_upper = 10.0

function len_constr(x)
    total_l = 0.0
    node4 = x
    node5 = [axis_node.x[1] + axis_node.x[1]-node4[1], node4[2]]
    for c in cells
        node_uid, node_vid = c.nodes
        node_u = collect(nodes[node_uid].x)
        if node_uid == 4
            node_u = node4
        elseif node_uid == 5
            node_u = node5
        end
        node_v = collect(nodes[node_vid].x)
        if node_vid == 4
            node_v = node4
        elseif node_vid == 5
            node_v = node5
        end
        total_l += LinearAlgebra.norm(node_u - node_v)
    end
    return total_l - length_upper
end

len_constr (generic function with 1 method)

In [49]:
len_constr([0.9, 1.0])

1.8639385588784432

In [50]:
using Nonconvex
import NLopt

alg = NLoptAlg(:LD_SLSQP)
options = NLoptOptions()

obj = sfl

# <= 0
# function constr_linear_1(x)
#     return 0.6*x[1] - x[2] + (0.6)
# end
# function constr_linear_2(x)
#     return -1.8*x[1]-1.0*x[2]
# end
# center = [1.0,1.0]
# radius = 0.1
# function constr3_ball(x)
#     return sum((x .- center).^2) - radius^2
# end

model = Nonconvex.Model(obj)
Nonconvex.addvar!(model, [0.0, -3.0], [2.0, 3.0])
add_ineq_constraint!(model, len_constr)
# add_ineq_constraint!(model, constr2)
# add_eq_constraint!(model, f)

alg = IpoptAlg()
options = IpoptOptions()
r = Nonconvex.optimize(model, alg, x0, options = options);

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [51]:
@show r.minimum
@show r.minimizer

r.minimum = 105.46741663072207
r.minimizer = [1.6742440683867743, -0.7665236799897839]


2-element Vector{Float64}:
  1.6742440683867743
 -0.7665236799897839